In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
import time
import os

# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize hand detector
detector = HandDetector(maxHands=1)

# Constants
offset = 20
imgSize = 500
baseFolder = "C:/Users/User/OneDrive/Documents/SignLanguageApp/SLangDataset/ownData/training"
letters = [chr(i) for i in range(ord('A'), ord('Z') + 1)]  # List of letters A-Z
captureInterval = 0.2  # Interval between captures in seconds

# Loop through each letter
for className in letters:
    print(f"Starting collection for class: {className}")
    folder = os.path.join(baseFolder, className)
    os.makedirs(folder, exist_ok=True)

    # Initialize counter for the current class
    counter = 0
    maxImages = 300

    collecting = False

    while counter < maxImages:
        success, img = cap.read()
        if not success:
            print("Failed to access camera.")
            break

        # Detect hands in the image
        hands, img = detector.findHands(img)
        if hands:
            hand = hands[0]
            x, y, w, h = hand['bbox']

            # Create a white canvas
            imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255

            # Crop the image around the detected hand
            imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]

            # Prevent errors from invalid crop boundaries
            if imgCrop.size == 0:
                continue

            aspectRatio = h / w

            try:
                if aspectRatio > 1:
                    # Height is greater than width
                    k = imgSize / h
                    wCal = math.ceil(k * w)
                    imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                    wGap = math.ceil((imgSize - wCal) / 2)
                    imgWhite[:, wGap:wCal + wGap] = imgResize
                else:
                    # Width is greater than height
                    k = imgSize / w
                    hCal = math.ceil(k * h)
                    imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                    hGap = math.ceil((imgSize - hCal) / 2)
                    imgWhite[hGap:hCal + hGap, :] = imgResize

                # Display the processed images
                cv2.imshow("ImageCrop", imgCrop)
                cv2.imshow("ImageWhite", imgWhite)
            except Exception as e:
                print(f"Error processing image: {e}")
                continue

            if collecting:
                # Save the image automatically
                counter += 1
                filePath = os.path.join(folder, f"{className.lower()}_{counter}.jpg")
                cv2.imwrite(filePath, imgWhite)
                print(f"Saved {counter}/{maxImages} images for class {className}")

        # Display the original image
        cv2.imshow("Image", img)

        # Check for key press
        key = cv2.waitKey(1)
        if key == ord('q'):
            collecting = True

        # Wait for the interval
        time.sleep(captureInterval)

    print(f"Completed collection for {className}.")
    input("Press 'Enter' to continue to the next letter.")

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize hand detector
detector = HandDetector(maxHands=1)

# Constants
offset = 20
imgSize = 500
baseFolder = "C:/Users/User/OneDrive/Documents/SignLanguageApp/SLangDataset/ownData/training"
captureInterval = 0.2  # Interval between captures in seconds

# Process only the letter 'Space'
className = 'Space'
print(f"Starting collection for class: {className}")
folder = os.path.join(baseFolder, className)
os.makedirs(folder, exist_ok=True)

# Initialize counter for the current class
counter = 0
maxImages = 300

collecting = False

while counter < maxImages:
    success, img = cap.read()
    if not success:
        print("Failed to access camera.")
        break

    # Detect hands in the image
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']

        # Create a white canvas
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255

        # Crop the image around the detected hand
        imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]

        # Prevent errors from invalid crop boundaries
        if imgCrop.size == 0:
            continue

        aspectRatio = h / w

        try:
            if aspectRatio > 1:
                # Height is greater than width
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap:wCal + wGap] = imgResize
            else:
                # Width is greater than height
                k = imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                hGap = math.ceil((imgSize - hCal) / 2)
                imgWhite[hGap:hCal + hGap, :] = imgResize

            # Display the processed images
            cv2.imshow("ImageCrop", imgCrop)
            cv2.imshow("ImageWhite", imgWhite)
        except Exception as e:
            print(f"Error processing image: {e}")
            continue

        if collecting:
            # Save the image automatically
            counter += 1
            filePath = os.path.join(folder, f"{className.lower()}_{counter}.jpg")
            cv2.imwrite(filePath, imgWhite)
            print(f"Saved {counter}/{maxImages} images for class {className}")

    # Display the original image
    cv2.imshow("Image", img)

    # Check for key press
    key = cv2.waitKey(1)
    if key == ord('q'):
        collecting = True

    # Wait for the interval
    time.sleep(captureInterval)

print(f"Completed collection for {className}.")
input("Press 'Enter' to exit.")

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

# Path to dataset
baseFolder = "C:/Users/User/OneDrive/Documents/SignLanguageApp/SLangDataset/asl-alphabet-2/training_annotated"
image_size = 224  # MobileNetV2 input size
batch_size = 4
num_classes = len(os.listdir(baseFolder))  # Number of classes (letters)

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,  # Use 20% of data for validation
)

# Load training data
train_generator = train_datagen.flow_from_directory(
    baseFolder,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
)

# Load validation data
val_generator = train_datagen.flow_from_directory(
    baseFolder,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
)

# Load the MobileNetV2 model
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Regularization
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation="softmax")(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
epochs = 30
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
)

# Save the model
model.save("sign_language_mobilenetv2.h5")

# Fine-tune the model (unfreeze some base model layers)
base_model.trainable = True
fine_tune_at = 100  # Unfreeze from this layer onwards

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5), loss="categorical_crossentropy", metrics=["accuracy"])

# Fine-tuning
fine_tune_epochs = 10
total_epochs = epochs + fine_tune_epochs
history_fine = model.fit(
    train_generator,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    validation_data=val_generator,
)

# Save the fine-tuned model
model.save("sign_language_mobilenetv2_fine_tuned.h5")


Found 1491 images belonging to 24 classes.
Found 364 images belonging to 24 classes.
Epoch 1/30
373/373 [==============================] - 10s 23ms/step - loss: 2.9724 - accuracy: 0.1395 - val_loss: 1.9860 - val_accuracy: 0.3791
Epoch 2/30
373/373 [==============================] - 8s 22ms/step - loss: 2.1552 - accuracy: 0.2991 - val_loss: 1.5588 - val_accuracy: 0.5000
Epoch 3/30
373/373 [==============================] - 8s 22ms/step - loss: 1.7991 - accuracy: 0.3930 - val_loss: 1.2230 - val_accuracy: 0.6071
Epoch 4/30
373/373 [==============================] - 8s 22ms/step - loss: 1.6216 - accuracy: 0.4272 - val_loss: 1.1145 - val_accuracy: 0.6374
Epoch 5/30
373/373 [==============================] - 8s 22ms/step - loss: 1.5207 - accuracy: 0.4628 - val_loss: 1.0385 - val_accuracy: 0.6786
Epoch 6/30
373/373 [==============================] - 8s 22ms/step - loss: 1.4463 - accuracy: 0.4903 - val_loss: 0.9313 - val_accuracy: 0.7143
Epoch 7/30
373/373 [==============================] - 8s

In [1]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
# classifier = Classifier("Model/keras_model.h5", "Model/labels.txt")
classifier = Classifier("C:/Users/User/OneDrive/Documents/SignLanguageApp/ModelCode/sign_language_mobilenetv2_fine_tuned.h5")
offset = 20
imgSize = 300
counter = 0
labels = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "Space"]
while True:
    success, img = cap.read()
    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255

        # Ensure the cropping boundaries are within the image dimensions
        y1, y2 = max(0, y - offset), min(img.shape[0], y + h + offset)
        x1, x2 = max(0, x - offset), min(img.shape[1], x + w + offset)
        imgCrop = img[y1:y2, x1:x2]

        imgCropShape = imgCrop.shape
        if imgCropShape[0] > 0 and imgCropShape[1] > 0:  # Ensure the cropped image is not empty
            aspectRatio = h / w
            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                imgResizeShape = imgResize.shape
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap:wCal + wGap] = imgResize
                prediction, index = classifier.getPrediction(imgWhite, draw=False)
                print(prediction, index)
            else:
                k = imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                imgResizeShape = imgResize.shape
                hGap = math.ceil((imgSize - hCal) / 2)
                imgWhite[hGap:hCal + hGap, :] = imgResize
                prediction, index = classifier.getPrediction(imgWhite, draw=False)
        if 0 <= index < len(labels):
            cv2.rectangle(imgOutput, (x - offset, y - offset-50),
                          (x - offset+90, y - offset-50+50), (255, 0, 255), cv2.FILLED)
            cv2.putText(imgOutput, labels[index], (x, y -26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
            cv2.rectangle(imgOutput, (x-offset, y-offset),
                          (x + w+offset, y + h+offset), (255, 0, 255), 4)
        cv2.imshow("ImageCrop", imgCrop)
        cv2.imshow("ImageWhite", imgWhite)
    cv2.imshow("Image", imgOutput)
    cv2.waitKey(1)

No Labels Found


AttributeError: 'NoneType' object has no attribute 'copy'